In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
data = pd.read_csv('Portfolio.csv', sep= ",")

In [4]:
data['Date'] = pd.to_datetime(data['Date'], format= '%d/%m/%y')

In [5]:
data.head()

,Date,FVX,GSPC,AMD,AMZN,GOOGL,MSFT,WMT
0,2008-01-10,2.821,968.750000,3.50,57.240002,179.859863,17.384617,43.533268
1,2008-01-11,1.944,896.239990,2.36,42.700001,146.626633,15.741920,43.587860
2,2008-01-12,1.551,903.250000,2.16,51.279999,153.978973,15.237188,43.728271
3,2009-01-01,1.873,825.880005,2.19,58.820000,169.434433,13.403088,36.913517
4,2009-01-02,2.020,735.090027,2.18,64.790001,169.164169,12.658468,38.574318


In [6]:
data.size

968

In [7]:
data = data.sort_values('Date')

In [8]:
returns = data[[key for key in dict(data.dtypes) if dict(data.dtypes)[key] in ('float64', 'int64') ]].pct_change()

In [9]:
returns.head()

,FVX,GSPC,AMD,AMZN,GOOGL,MSFT,WMT
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.310883,-0.074849,-0.325714,-0.254018,-0.184773,-0.094491,0.001254
2,-0.202160,0.007822,-0.084746,0.200937,0.050143,-0.032063,0.003221
3,0.207608,-0.085657,0.013889,0.147036,0.100374,-0.120370,-0.155843
4,0.078484,-0.109931,-0.004566,0.101496,-0.001595,-0.055556,0.044992


In [10]:
returns = returns[1:]
returns.head()

,FVX,GSPC,AMD,AMZN,GOOGL,MSFT,WMT
1,-0.310883,-0.074849,-0.325714,-0.254018,-0.184773,-0.094491,0.001254
2,-0.202160,0.007822,-0.084746,0.200937,0.050143,-0.032063,0.003221
3,0.207608,-0.085657,0.013889,0.147036,0.100374,-0.120370,-0.155843
4,0.078484,-0.109931,-0.004566,0.101496,-0.001595,-0.055556,0.044992
5,-0.170792,0.085404,0.399083,0.133508,0.029794,0.145261,0.058083


In [11]:
stockNames = list(data)[1:]
n  = len(stockNames)

In [12]:
returns.head()

,FVX,GSPC,AMD,AMZN,GOOGL,MSFT,WMT
1,-0.310883,-0.074849,-0.325714,-0.254018,-0.184773,-0.094491,0.001254
2,-0.202160,0.007822,-0.084746,0.200937,0.050143,-0.032063,0.003221
3,0.207608,-0.085657,0.013889,0.147036,0.100374,-0.120370,-0.155843
4,0.078484,-0.109931,-0.004566,0.101496,-0.001595,-0.055556,0.044992
5,-0.170792,0.085404,0.399083,0.133508,0.029794,0.145261,0.058083


In [13]:
#Linear programming
#!pip install cvxopt
import cvxopt as opt #convex optimizer

#Minimize Loss
#Subject to Portfolio Weight = 1
#Weights are non negative
#Return >= Expected return

In [14]:
from cvxopt import matrix,solvers

In [15]:
expectedReturn = np.mean(returns)
maxLoss = np.min(returns)

In [153]:
expectedReturn

<7x1 matrix, tc='d'>

In [17]:
expectedReturn = matrix(expectedReturn)

In [18]:
maxLoss

FVX     -0.310883
GSPC    -0.109931
AMD     -0.391691
AMZN    -0.254018
GOOGL   -0.184773
MSFT    -0.155206
WMT     -0.155843
dtype: float64

In [85]:
#We seek to minimize the maximum loss of our portfolio
risk = -1*maxLoss #converting to to positive value

In [138]:
risk.head()

FVX      0.310883
GSPC     0.109931
AMD      0.391691
AMZN     0.254018
GOOGL    0.184773
dtype: float64

In [139]:
objective = matrix(risk)

In [140]:
constraintEqualityLhs = matrix(1.0, (1,n)) #total portfolio percentage - 100%

In [141]:
constraintEqualityLhs

<1x7 matrix, tc='d'>

In [142]:
constraintEqualityRhs = matrix(1.0)

In [143]:
constraintInequalityLhsRow1 = matrix(-np.identity(n))  #non negativity constraint

In [144]:
constraintInequalityRhsRow1 = matrix(0.0, (n,1)) #n rows with 1 column in which every value equal 0 

In [145]:
constraintInequalityLhsRow2 = matrix(-np.transpose(np.array(expectedReturn)))

In [146]:
constraintInequalityRhsRow2 = matrix(-np.ones((1,1))*0.02) #(greater than 2% expected return)

In [147]:
constraintInEqualityLhscombined = matrix(np.concatenate((constraintInequalityLhsRow1, constraintInequalityLhsRow2),0))

In [148]:
constraintInEqualityRhscombined = matrix(np.concatenate((constraintInequalityRhsRow1, constraintInequalityRhsRow2),0))

In [149]:
solution = solvers.lp(objective, constraintInEqualityLhscombined, constraintInEqualityRhscombined, constraintEqualityLhs, constraintEqualityRhs)

     pcost       dcost       gap    pres   dres   k/t
 0:  2.2320e-01  2.4546e-01  1e+01  3e+00  3e+00  1e+00
 1:  1.8866e-01  7.8911e-02  5e-01  3e-01  3e-01  4e-03
 2:  1.5071e-01  1.3485e-01  6e-02  6e-02  6e-02  3e-03
 3:  1.3212e-01  1.2621e-01  3e-02  2e-02  2e-02  2e-03
 4:  1.7453e-01  1.7250e-01  6e-02  2e-02  2e-02  5e-03
 5:  1.6857e-01  1.6836e-01  7e-03  2e-03  3e-03  6e-04
 6:  1.7105e-01  1.7103e-01  8e-04  3e-04  3e-04  8e-05
 7:  1.7107e-01  1.7106e-01  4e-04  1e-04  1e-04  4e-05
 8:  1.7122e-01  1.7122e-01  5e-05  2e-05  2e-05  5e-06
 9:  1.7122e-01  1.7122e-01  5e-07  2e-07  2e-07  6e-08
10:  1.7122e-01  1.7122e-01  5e-09  2e-09  2e-09  6e-10
Optimal solution found.


In [150]:
weightsLinear = solution['x']

In [151]:
weightsLinear = np.array(weightsLinear)

In [152]:
weightsLinear

array([[  2.16392849e-09],
       [  1.06393816e-06],
       [  2.87482253e-09],
       [  1.62059004e-01],
       [  2.00792420e-08],
       [  8.37939898e-01],
       [  8.89739496e-09]])

In [134]:
np.sort(weightsLinear)

array([[  2.16392893e-09],
       [  1.06393819e-06],
       [  2.87482298e-09],
       [  1.62059004e-01],
       [  2.00792428e-08],
       [  8.37939898e-01],
       [  8.89739555e-09]])

In [126]:
np.max(weightsLinear)

0.83793989838187344

In [113]:
sum(weightsLinear)

array([ 1.])

In [106]:
portfolioRisk = np.dot(weightsLinear.T, risk)

In [108]:
portfolioRisk
#worst case scenario , we will lose 17% (conservative estimate)

array([ 0.17121958])

In [154]:
portfolioReturn = np.dot(expectedReturn.T, weightsLinear)

In [155]:
portfolioReturn

array([[ 0.02]])

In [ ]:
#Change risk to be portfolio variance (rathen than sum of maxloss of each stock)
#Variance of a portfolio is found using covariance matrix
#Objective function has quadratic terms
#Quadratic programming

In [157]:
#Integer programming - cannot use convex optimizer directly
from cvxopt import glpk

In [ ]:
constraintInequalityLhsRow2 = matrix(-np.transpose(np.array(expectedReturn)))

In [ ]:
constraintInequalityRhsRow2 = matrix(-np.ones((1,1))*0.02) #(greater than 2% expected return)